In [ ]:
import numpy as np
import widgyts
import ipydatawidgets
import ipywidgets
import traitlets
from IPython.display import DisplayObject, display
import yt
import h5py

First we'll load IsolatedGalaxy, one of the datasets that is used throughout yt's documentation. 

In [ ]:
ds = yt.load("/Users/madicken/repos/talks/2019-03-13-pydata/notebooks/data/IsolatedGalaxy/galaxy0030/galaxy0030")

### Using Jupyter Widgets out-of-the-box With yt

In [ ]:
ds = yt.load("/Users/madicken/repos/talks/2019-03-13-pydata/notebooks/data/IsolatedGalaxy/galaxy0030/galaxy0030")
class State(traitlets.HasTraits):
    sl = traitlets.Instance(yt.data_objects.selection_data_containers.YTSlice)
    im = traitlets.Instance(ipywidgets.Image)
    width = traitlets.Float(1.0)
    center_x = traitlets.Float(0.5)
    center_y = traitlets.Float(0.5)
    frb = traitlets.Instance(yt.FixedResolutionBuffer)
    
    @traitlets.observe('center_x', 'center_y')
    def on_change_center(self, event):
        self.frb = self.sl.to_frb(self.width, (512, 512), center=(self.center_x, self.center_y))
        
    @traitlets.observe('width')
    def on_change_width(self, event):
        self.frb = self.sl.to_frb(self.width, (512, 512), center=(self.center_x, self.center_y))
    
    @traitlets.observe('frb')
    def on_change_frb(self, event):
        self.im.value = yt.write_bitmap(yt.write_image(np.log10(self.frb["density"]), None), None)
        
# ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030")
im = ipywidgets.Image(width = 512, height = 512)
s = State(sl = ds.r[:,:,0.5], im = im)
width_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, value=1.0, step=0.001)
center_x_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, value=0.5, step=0.001)
center_y_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, value=0.5, step=0.001)
traitlets.link((width_slider, 'value'), (s, 'width'))
traitlets.link((center_x_slider, 'value'), (s, 'center_x'))
traitlets.link((center_y_slider, 'value'), (s, 'center_y'))

display(ipywidgets.VBox([im, width_slider, center_x_slider, center_y_slider]))

### Using widgyts on the same dataset

In [ ]:
s = ds.r[:,:,0.5]
frb = widgyts.FRBViewer(height = 1024, width = 1024,
                            px = s["px"], py = s["py"], pdx = s["pdx"], pdy = s["pdy"],
                            val = s["density"], layout=ipywidgets.Layout(width = '500px', height='500px'))
controls = frb.setup_controls()
display(ipywidgets.HBox([controls,frb]))

### Slice from Pop II prime through the densest location 

In [ ]:
with h5py.File("/Users/madicken/repos/talks/2019-03-13-pydata/notebooks/data/DD0182_slice.h5", "r") as f:
    s = {}
    for field in ("px", "py", "pdx", "pdy", "density", "temperature"):
        print("Reading ", field)
        s[field] = f[field][:]
frb_dens = widgyts.FRBViewer(height = 512, width = 512,
                            px = s["px"], py = s["py"], pdx = s["pdx"], pdy = s["pdy"],
                            val = s["density"],
                            layout = ipywidgets.Layout(width = '500px', height='500px'))
controls = frb_dens.setup_controls()
display(ipywidgets.HBox([controls,frb_dens]))

#### We can also link our widget traitlets to explore multiple fields at the same time. 

first let's add another zoom widget with a log scale. 

In [ ]:
frb_dens.colormaps.is_log = True
width_slider = ipywidgets.FloatLogSlider(value = 0.0, max = 0.0, min = np.log10(512 * s["pdx"].min()), base = 10)

width_slider.value = 1.0
def on_changed(event):
    frb_dens.view_width = (event['new'], event['new'])
width_slider.observe(on_changed, 'value', 'change')
width_slider

In [ ]:
frb_temp = widgyts.FRBViewer(height = 512, width = 512,
                            px = s["px"], py = s["py"], pdx = s["pdx"], pdy = s["pdy"],
                            val = s["temperature"], layout=ipywidgets.Layout(width = '500px', height='500px'))
ipywidgets.link((frb_dens, 'view_center'), (frb_temp, 'view_center'))
ipywidgets.link((frb_dens, 'view_width'), (frb_temp, 'view_width'))
ipywidgets.link((frb_dens.colormaps, 'is_log'), (frb_temp.colormaps, 'is_log'))
display(ipywidgets.VBox([width_slider,ipywidgets.HBox([controls,frb_dens, frb_temp])]))

frb_temp.colormaps.map_name='inferno'
frb_temp.colormaps.min_val=5.874986614501054
frb_temp.colormaps.max_val=15653.545441919285

#### We can also access the temperature field properties through the traitlets. 

In [ ]:
frb_temp.colormaps.map_name='doom'

### Projection of Pop II prime dataset

In [ ]:
with h5py.File("/Users/madicken/repos/talks/2019-03-13-pydata/notebooks/data/DD0182_proj.h5", "r") as f:
    s = {}
    for field in ("px", "py", "pdx", "pdy", "density", "temperature"):
        s[field] = f[field][:]

# putting the fields into two different FRB views
frb_projd = widgyts.FRBViewer(height = 512, width = 512,
                            px = s["px"], py = s["py"], pdx = s["pdx"], pdy = s["pdy"],
                            val = s["density"], layout=ipywidgets.Layout(width = '500px', height='500px'))
frb_projt = widgyts.FRBViewer(height = 512, width = 512,
                            px = s["px"], py = s["py"], pdx = s["pdx"], pdy = s["pdy"],
                            val = s["temperature"], layout=ipywidgets.Layout(width = '500px', height='500px'))

# linking the views so their windows are aligned
ipywidgets.link((frb_projd, 'view_center'), (frb_projt, 'view_center'))
ipywidgets.link((frb_projd, 'view_width'), (frb_projt, 'view_width'))
ipywidgets.link((frb_projd.colormaps, 'is_log'), (frb_projt.colormaps, 'is_log'))

#adding in the log slider 
width_slider2 = ipywidgets.FloatLogSlider(value = 0.0, max = 0.0, min = np.log10(512 * s["pdx"].min()), base = 10)
width_slider2.value = 1.0
def on_changed(event):
    frb_projd.view_width = (event['new'], event['new'])
width_slider2.observe(on_changed, 'value', 'change')

#adding in the controls
controls = frb_projd.setup_controls()
display(ipywidgets.VBox([width_slider2,ipywidgets.HBox([controls,frb_projd, frb_projt])]))

# setting bounds of the temperature plot
frb_projt.colormaps.map_name='inferno'
frb_projt.colormaps.min_val=8.5
frb_projt.colormaps.max_val=1283

# setting the log scale
frb_projd.colormaps.is_log = True
frb_projd.colormaps.max_val = 1e-24

In [ ]:
frb_projt.colormaps.map_name='inferno'